In [1]:
'''
1. データセットの読み込みと正規化
'''
# tensorflow.keras のインポート
from tensorflow import keras

# Fashion-MNISTデータセットの読み込み
(x_train, t_train), (x_test, t_test) = keras.datasets.fashion_mnist.load_data()

# 訓練データを正規化
x_train = x_train.astype('float32') / 255
# テストデータを正規化
x_test =  x_test.astype('float32') / 255

In [2]:
'''
2.モデルの作成
'''
class RNN(keras.Model):
    '''畳み込みニューラルネットワーク
    
    Attributes:
      i(Conv2D): 入力層
      l1(Conv2D): LSTM層1
      l2(Conv2D): LSTM層2
      l3(Conv2D): LSTM層3
      o(Dense): 出力層
    '''
    def __init__(self, output_dim):
        '''
        Parameters:
          output_dim(int): 出力層のユニット数(次元)
                  '''
        super().__init__()
        
        # 入力層
        self.i = keras.layers.InputLayer(
            input_shape=(28,28)            # 入力の形状は(28,28)
            )
        
        #  LSTM（ユニット数＝128）
        weight_decay = 1e-4  # ハイパーパラメーター
        self.l1 = keras.layers.LSTM(units=128, dropout=0.25, return_sequences=True) 
        self.l2 = keras.layers.LSTM(units=128, dropout=0.25, return_sequences=True)
        self.l3 = keras.layers.LSTM(
            units=128, dropout=0.5, return_sequences=False,
            kernel_regularizer=keras.regularizers.l2(weight_decay)) # 正則化

        # 出力層
        self.o =  keras.layers.Dense(units=10, # ニューロン数は10
          activation='softmax')                # 活性化はソフトマックス関数
        
        # すべての層をリストにする
        self.ls = [self.i, self.l1, self.l2, self.l3, self.o]

    def call(self, x):
        '''MLPのインスタンスからコールバックされる関数
        
        Parameters: x(ndarray(float32)):訓練データ、または検証データ
        Returns(float32): RNNの出力として要素数10の1階テンソル        
        '''
        for layer in self.ls:
            x = layer(x)
        
        return x

In [3]:
'''
3.損失関数の定義
'''
# 損失関数はスパースラベル対応クロスエントロピー誤差
cce = keras.losses.SparseCategoricalCrossentropy()

def loss(t, y):
    '''損失関数
    Parameters: t(ndarray(float32)):正解ラベル
                y(ndarray(float32)):予測値
                
    Returns: クロスエントロピー誤差
    '''
    return cce(t, y)

In [4]:
'''
4.オプティマイザーのオブジェクトの生成
  損失と精度を測定するオブジェクトの生成
'''
# 勾配降下アルゴリズムを使用するオプティマイザーを生成
optimizer = keras.optimizers.SGD(learning_rate=0.1)

# 損失を記録するオブジェクトを生成
train_loss = keras.metrics.Mean()
# 精度を記録するオブジェクトを生成
train_acc = keras.metrics.SparseCategoricalAccuracy()
# 検証時の損失を記録するオブジェクトを生成
val_loss = keras.metrics.Mean()
# 検証時の精度を記録するオブジェクトを生成
val_acc = keras.metrics.SparseCategoricalAccuracy()

In [5]:
'''
5.勾配降下アルゴリズムによるパラメーターの更新処理
'''
import tensorflow as tf

def train_step(x, t):
    '''学習を1回行う
    
    Parameters: x(ndarray(float32)):訓練データ
                t(ndarray(float32)):正解ラベル
                
    Returns:
      ステップごとのクロスエントロピー誤差
    '''
    # 自動微分による勾配計算を記録するブロック
    with tf.GradientTape() as tape:
        # モデルに入力して順伝搬の出力値を取得
        outputs = model(x)
        # 出力値と正解ラベルの誤差
        tmp_loss = loss(t, outputs)
        
    # tapeに記録された操作を使用して誤差の勾配を計算        
    grads = tape.gradient(
        # 現在のステップの誤差
        tmp_loss,
        # バイアス、重みのリストを取得
        model.trainable_variables)
    # 勾配降下法の更新式を適用してバイアス、重みを更新
    optimizer.apply_gradients(zip(grads,
                                  model.trainable_variables))
    
    # 損失をMeanオブジェクトに記録
    train_loss(tmp_loss)
    # 精度をCategoricalAccuracyオブジェクトに記録
    train_acc(t, outputs)

In [6]:
'''
6. 検証データによる評価を行う関数
'''
def val_step(x, t):
    '''検証データをモデルに入力して損失と精度を測定
    
    Parameters: x(ndarray(float32)):検証データ
                t(ndarray(float32)):正解ラベル
    '''
    # 検証データの予測値を取得
    preds = model(x)
    # 出力値と正解ラベルの誤差
    tmp_loss = loss(t, preds)
    # 損失をMeanオブジェクトに記録
    val_loss(tmp_loss)
    # 精度をCategoricalAccuracyオブジェクトに記録
    val_acc(t, preds)

In [7]:
'''
7. 訓練データの20パーセントのデータを検証用のデータにする
'''
from sklearn.model_selection import train_test_split

# 訓練データと検証データに8：2の割合で分割  \は行継続文字
x_train, x_val, t_train, t_val = \
    train_test_split(x_train, t_train, test_size=0.2)

In [8]:
%%time
'''
8.モデルを生成して学習する
'''
from sklearn.utils import shuffle

# エポック数
epochs = 100
# ミニバッチのサイズ
batch_size = 64
# 訓練データのステップ数
steps = x_train.shape[0] // batch_size
# 検証データのステップ数
steps_val = x_val.shape[0] // batch_size

# 出力層10ニューロンのモデルを生成
model = RNN(10)

# 学習を行う
for epoch in range(epochs):
    # 訓練データと正解ラベルをシャッフル
    x_, t_ = shuffle(x_train, t_train, random_state=1)
    
    # 1ステップにおけるミニバッチを使用した学習
    for step in range(steps):
        start = step * batch_size # ミニバッチの先頭インデックス
        end = start + batch_size  # ミニバッチの末尾のインデックス
        # ミニバッチでバイアス、重みを更新
        train_step(x_[start:end], t_[start:end])
        
    # 検証データによるモデルの評価
    for step_val in range(steps_val):
        start = step_val * batch_size # ミニバッチの先頭インデックス
        end = start + batch_size      # ミニバッチの末尾のインデックス
        # 検証データのミニバッチで損失と精度を測定
        val_step(x_[start:end], t_[start:end])

    # 1エポックごとに結果を出力
    print('epoch({}) train_loss: {:.4} train_acc: {:.4}'
          'val_loss: {:.4} val_acc: {:.4}'.format(
              epoch+1,
              train_loss.result(), # 訓練データの損失を出力
              train_acc.result(),  # 訓練データの精度を出力
              val_loss.result(),   # 検証データの損失を出力
              val_acc.result()     # 検証データの精度を出力
              ))

epoch(1) train_loss: 0.8413 train_acc: 0.6916val_loss: 0.5617 val_acc: 0.7894
epoch(2) train_loss: 0.6929 train_acc: 0.7434val_loss: 0.5227 val_acc: 0.8028
epoch(3) train_loss: 0.6202 train_acc: 0.7695val_loss: 0.4914 val_acc: 0.8145
epoch(4) train_loss: 0.5735 train_acc: 0.786val_loss: 0.4685 val_acc: 0.8235
epoch(5) train_loss: 0.5395 train_acc: 0.7984val_loss: 0.4512 val_acc: 0.8304
epoch(6) train_loss: 0.5128 train_acc: 0.8085val_loss: 0.4357 val_acc: 0.837
epoch(7) train_loss: 0.491 train_acc: 0.8167val_loss: 0.4227 val_acc: 0.8424
epoch(8) train_loss: 0.4727 train_acc: 0.8237val_loss: 0.4115 val_acc: 0.8469
epoch(9) train_loss: 0.457 train_acc: 0.8295val_loss: 0.4015 val_acc: 0.8507
epoch(10) train_loss: 0.4432 train_acc: 0.8347val_loss: 0.3928 val_acc: 0.854
epoch(11) train_loss: 0.4309 train_acc: 0.8393val_loss: 0.3848 val_acc: 0.857
epoch(12) train_loss: 0.4199 train_acc: 0.8434val_loss: 0.3772 val_acc: 0.8597
epoch(13) train_loss: 0.4097 train_acc: 0.8471val_loss: 0.3705 val_

In [9]:
'''
9. テストデータによるモデルの評価
'''
# 損失を記録するオブジェクトを生成
test_loss = keras.metrics.Mean()
# 精度を記録するオブジェクトを生成
test_acc = keras.metrics.SparseCategoricalAccuracy()

def test_step(x, t):
    '''テストデータをモデルに入力して損失と精度を測定
    
    Parameters: x(ndarray(float32)):テストデータ
                t(ndarray(float32)):正解ラベル
    '''
    # テストデータの予測値を取得
    preds = model(x)
    # 出力値と正解ラベルの誤差
    tmp_loss = loss(t, preds)
    # 損失をMeanオブジェクトに記録
    test_loss(tmp_loss)
    # 精度をCategoricalAccuracyオブジェクトに記録
    test_acc(t, preds)

# テストデータで予測して損失と精度を取得
test_step(x_test, t_test)

print('test_loss: {:.4f}, test_acc: {:.4f}'.format(
    test_loss.result(),
    test_acc.result()
))

test_loss: 0.6401, test_acc: 0.8814
